# From Detection to Credibility: A Machine Learning Framework for Assessing News Source Reliability



**Motivation**

As media continues to grow in volume, it is becoming increasingly difficult to differentiate real and fake news effectively. It is thus imperative for us to find better ways to identify fake news, and for us, this means with the help of data mining and machine learning.

In the first part of our project, we will focus on experimenting with different data processing techniques and predictive models, optimising our final pipeline and model to accurately identify fake news.

For the second part, we want to apply our trained model to scraped news data from popular US media outlets and access the credibility of these media outlets. This way we can help the public to make more informed decisions about what media outlets they can trust. 


## 2nd Part: Fake News Classification Use Case

For the second part, we scraped articles from 10 different news sites, split into two categories of news sites. 

The dimensions are shown below:
- **Index:**: Index.
- **title:** Title of news article.
- **text:** Text content of news article.
- **label:** Whether news article is real (0) or fake (1).

The Fake News Dataset is split into 3 `csv` files (`part1.csv`, `part2.csv`, `part3.csv`) so that size does not exceed size limit to push changes to GitHub.

## Import Libraries

Please uncomment the code box below to pip install relevant dependencies for this notebook.

In [118]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm

# Plotting and Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Language Detection packages
# `langdetect` for detecting language
from langdetect import detect as langdetect_detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
# `langid` for an alternative language detection method
from langid import classify as langid_classify

# Text Preprocessing and NLP
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords
# Tokenizing sentences/words
from nltk.tokenize import word_tokenize
# Part-of-speech tagging
from nltk import pos_tag
# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer
import nltk
# Regular expressions for text pattern matching
import re

# Word Cloud generation
from wordcloud import WordCloud

In [119]:
## Unreliable websites
breitbart_df = pd.read_csv('./unreliable websites/breitbart_articles.csv')
dailycaller_df = pd.read_csv('./unreliable websites/dailycaller_articles.csv')
naturalnews_df = pd.read_csv('./unreliable websites/naturalnews_articles.csv')
newsmax_df = pd.read_csv('./unreliable websites/newsmax_articles.csv')
zerohedge_df = pd.read_csv('./unreliable websites/zerohedge_articles.csv')

# Reliable websites
cnn_df = pd.read_csv('./reliable websites/cnn_articles.csv')
ap_df = pd.read_csv('./reliable websites/ap_articles.csv')
bbc_df = pd.read_csv('./reliable websites/bbc_articles.csv')
npr_df = pd.read_csv('./reliable websites/npr_articles.csv')
guardian_df = pd.read_csv('./reliable websites/guardian_articles.csv')

## Combine all 10 dataframes into 1
Here we combine all 10 dataframes into 1 dataframe (`data_raw`) by concatenating along rows.

We also reset the `index` and drop the old `index` column.

In [120]:
# Combining all 3 dataframes into 1
data_raw = pd.concat([breitbart_df, dailycaller_df, naturalnews_df,newsmax_df,zerohedge_df,cnn_df,ap_df,bbc_df,npr_df,guardian_df], axis=0)

# Reset index and drop old index column
data_raw = data_raw.reset_index(drop=True)

data_raw.info()
print("Dataframe Shape: ", data_raw.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955 entries, 0 to 954
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   source   955 non-null    object
 1   title    955 non-null    object
 2   content  955 non-null    object
 3   date     955 non-null    object
dtypes: object(4)
memory usage: 30.0+ KB
Dataframe Shape:  (955, 4)


In [121]:
data_raw.head()

,source,title,content,date
0,Breitbart,Russia Practices 'Massive Nuclear Strike' in A...,“Important to have modern and constantly ready...,30/10/2024
1,Breitbart,Nolte: Trump and Republican Senate Challenger ...,The final Insider Advantage poll out of Wiscon...,29/10/2024
2,Breitbart,Sen. Tom Cotton Barnstorms Nation Campaigning ...,Sen. Tom Cotton (R-AR) is putting in long hour...,29/10/2024
3,Breitbart,U.S. Marines Successfully Test Israel's Iron D...,The U.S. Marines have successfully tested a mo...,30/10/2024
4,Breitbart,"Seoul, Japan Warn North Korea Preparing Nuclea...",The South Korean Defense Intelligence Agency (...,30/10/2024


In [122]:
data_raw['source'].value_counts()

source
The Daily Caller    100
Natural News        100
Zerohedge           100
Breitbart            99
CNN                  99
Guardian             99
AP                   97
News Max             95
NPR                  86
BBC                  80
Name: count, dtype: int64

## Selection Criteria
- Language: Only use articles written in English (using language detection if necessary).
- Date Range: Focus on articles published in 2024 or, at most, 2023 to ensure credibility remains up-to-date.
- Content Focus: Only articles from the politics/election sections of each news website, targeting US election-related topics.

### Additional Filtering
- Define Political Content: Articles should contain keywords like “election,” “government,” “policy,” or “candidate” to qualify as political.
- Exclude Advertisements: Filter out content with commercial keywords or phrases related to advertisements.
- Exclude Opinion Pieces: Identify and exclude articles labeled as “opinion,” “editorial,” or similar terms, or those appearing in “Opinion” sections.
- Remove Outliers by Length: Filter out articles with fewer than 100 words or more than 5,000 words to focus on substantial content.
- Regex Filtering: Use regular expressions to remove boilerplate or irrelevant sections, such as “All rights reserved,” “Read more,” bylines, and embedded links to other articles or advertisements.

#### Step 1: Language Detection

In [123]:
# 1) Set a seed for langdetect to ensure reproducibility
DetectorFactory.seed = 0

# 2a) Simplified preprocessing: only remove non-alphabetic characters
def preprocess_text_simple(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    return text.strip()

# 2b) Check if the text is non-language (e.g., numbers, symbols only)
def is_non_language_text(text):
    if re.match(r'^[^a-zA-Z]*$', text):  # Check if text has no alphabetic characters
        return True
    return False

# 3a) Function to get langdetect prediction
def get_langdetect_prediction(text):
    try:
        # Directly use text without preprocessing for efficiency
        if len(text) < 10 or is_non_language_text(text):
            return "unknown"
        lang = langdetect_detect(text)
        return lang
    except LangDetectException:
        return "unknown"

# 3b) Function to get langid prediction
def get_langid_prediction(text):
    try:
        lang, _ = langid_classify(text)
        if len(text) < 10 or is_non_language_text(text):
            return "unknown"
        return lang
    except Exception:
        return "unknown"

# 4) Function to calculate majority vote for each language
def calculate_majority_vote(predictions):
    vote_counts = {}
    for lang in predictions:
        if lang in vote_counts:
            vote_counts[lang] += 1
        else:
            vote_counts[lang] = 1
    return vote_counts

# 5) Parallel processing for efficiency with limited workers
def parallel_detection(text):
    with ThreadPoolExecutor(max_workers=4) as executor:
        results = list(executor.map(lambda func: func(text), 
                                    [get_langdetect_prediction, get_langid_prediction]))
    return results

# 6) Caching function for repeated inputs
@lru_cache(maxsize=500)
def get_cached_language(text):
    return combined_language_detection(text)

# 7) Combined majority voting language detection function
def combined_language_detection(text):
    # Check if the text is non-language (e.g., numbers, symbols only)
    if is_non_language_text(text):
        return "unknown"
    
    # Run the detectors in parallel for efficiency
    predictions = parallel_detection(text)
    
    # Calculate majority vote for each language based on predictions
    vote_counts = calculate_majority_vote(predictions)
    
    # Determine the language with the highest majority vote
    final_language = max(vote_counts, key=vote_counts.get)
    
    # If "unknown" is the most common or if all detectors fail, return "unknown"
    if final_language == "unknown" or vote_counts[final_language] <= 1:
        return "unknown"
    
    return final_language

# 8) Apply the cached function to each text in the DataFrame with a progress bar
data_raw['language'] = [get_cached_language(text) for text in tqdm(data_raw['content'], desc="Language Detection")]

# 9) Display the DataFrame with detected languages
data_raw

Language Detection: 100%|██████████| 955/955 [00:08<00:00, 118.88it/s]


,source,title,content,date,language
0,Breitbart,Russia Practices 'Massive Nuclear Strike' in A...,“Important to have modern and constantly ready...,30/10/2024,en
1,Breitbart,Nolte: Trump and Republican Senate Challenger ...,The final Insider Advantage poll out of Wiscon...,29/10/2024,en
2,Breitbart,Sen. Tom Cotton Barnstorms Nation Campaigning ...,Sen. Tom Cotton (R-AR) is putting in long hour...,29/10/2024,en
3,Breitbart,U.S. Marines Successfully Test Israel's Iron D...,The U.S. Marines have successfully tested a mo...,30/10/2024,en
4,Breitbart,"Seoul, Japan Warn North Korea Preparing Nuclea...",The South Korean Defense Intelligence Agency (...,30/10/2024,en
...,...,...,...,...,...
950,Guardian,How can the candidate with most votes lose? Th...,Even though the United States touts its status...,19/10/2024,en
951,Guardian,The RBA will likely hold interest rates – but ...,When the Reserve Bank board meet next week to ...,30/10/2024,en
952,Guardian,Trump’s mass deportation plan would be ‘econom...,"If elected, Donald Trump plans to carry out “t...",30/10/2024,en
953,Guardian,Sun belt to October surprise: US election term...,"On Tuesday 5 November, Americans will vote aft...",10/10/2024,en


In [124]:
# Drop rows where language is NOT in english and reset the index
data_raw = data_raw[data_raw['language'] == 'en'].reset_index(drop=True)
data_raw

,source,title,content,date,language
0,Breitbart,Russia Practices 'Massive Nuclear Strike' in A...,“Important to have modern and constantly ready...,30/10/2024,en
1,Breitbart,Nolte: Trump and Republican Senate Challenger ...,The final Insider Advantage poll out of Wiscon...,29/10/2024,en
2,Breitbart,Sen. Tom Cotton Barnstorms Nation Campaigning ...,Sen. Tom Cotton (R-AR) is putting in long hour...,29/10/2024,en
3,Breitbart,U.S. Marines Successfully Test Israel's Iron D...,The U.S. Marines have successfully tested a mo...,30/10/2024,en
4,Breitbart,"Seoul, Japan Warn North Korea Preparing Nuclea...",The South Korean Defense Intelligence Agency (...,30/10/2024,en
...,...,...,...,...,...
949,Guardian,How can the candidate with most votes lose? Th...,Even though the United States touts its status...,19/10/2024,en
950,Guardian,The RBA will likely hold interest rates – but ...,When the Reserve Bank board meet next week to ...,30/10/2024,en
951,Guardian,Trump’s mass deportation plan would be ‘econom...,"If elected, Donald Trump plans to carry out “t...",30/10/2024,en
952,Guardian,Sun belt to October surprise: US election term...,"On Tuesday 5 November, Americans will vote aft...",10/10/2024,en


In [125]:
data_raw['language'].value_counts()

# Drop the  'language' column
data_raw = data_raw.drop(columns=['language'])

data_raw.head()

,source,title,content,date
0,Breitbart,Russia Practices 'Massive Nuclear Strike' in A...,“Important to have modern and constantly ready...,30/10/2024
1,Breitbart,Nolte: Trump and Republican Senate Challenger ...,The final Insider Advantage poll out of Wiscon...,29/10/2024
2,Breitbart,Sen. Tom Cotton Barnstorms Nation Campaigning ...,Sen. Tom Cotton (R-AR) is putting in long hour...,29/10/2024
3,Breitbart,U.S. Marines Successfully Test Israel's Iron D...,The U.S. Marines have successfully tested a mo...,30/10/2024
4,Breitbart,"Seoul, Japan Warn North Korea Preparing Nuclea...",The South Korean Defense Intelligence Agency (...,30/10/2024


In [ ]:
#  Copy data for safety
data_filtered = data_raw.copy()
print("Number of articles before filtering:", len(data_filtered))

# Step 2: Political Keywords Filter
us_politics_keywords = [
    # Core Keywords
    "election", "elections", "2024 election", "presidential election", "campaign", "campaigning",
    "primary", "primaries", "polling", "polls",

    # Key Political Figures
    "Kamala Harris", "Donald Trump", "Joe Biden", "Ron DeSantis", "Gavin Newsom", 
    "Mike Pence", "Vivek Ramaswamy", "Tim Walz"

    # Political Parties & Groups
    "Democrat", "Democratic Party", "Republican", "GOP", "Republican Party",
    "Independent", "Third party", "PAC", "Super PAC",

    # Political Issues & Controversies
    "voting rights", "voter suppression", "absentee ballot", "mail-in ballot",
    "Electoral College", "Supreme Court", "abortion", "Roe v. Wade", "gun control",
    "Second Amendment", "immigration", "border security", "healthcare", 
    "Medicare", "Affordable Care Act", "climate change", "Green New Deal", 
    "inflation", "economic policy", "tax cuts", "tax reform", "foreign policy", 
    "foreign relations",

    # U.S. Government Bodies & Offices
    "Congress", "Senate", "Senators", "House of Representatives", "White House",
    "Supreme Court", "Federal government", "State government",

    # Policies & Bills
    "voting reform", "healthcare reform", "climate policy", "gun legislation", 
    "economic recovery", "infrastructure bill", "Social Security", "student loan forgiveness",

    # Social & Cultural Issues
    "social justice", "racial equality", "police reform", "civil rights",
    "freedom of speech", "religious freedom",

    # Election Processes
    "debates", "presidential debate", "swing state", "battleground state",
    "electoral votes",

    # Additional Relevant Terms
    "approval rating", "national convention", "lobbying", "lobbyist",
    "scandal", "investigation", "political rally", "rally"
]
data_filtered = data_filtered[data_filtered['content'].str.contains('|'.join(us_politics_keywords), case=False, na=False)]
print("Number of articles after political keyword filtering:", len(data_filtered))

# Step 3: Exclude Opinion Pieces (assuming titles or content contain specific indicators of opinion)
opinion_keywords = ['opinion', 'editorial', 'op-ed']
data_filtered = data_filtered[~data_filtered['title'].str.contains('|'.join(opinion_keywords), case=False, na=False)]
data_filtered = data_filtered[~data_filtered['content'].str.contains('|'.join(opinion_keywords), case=False, na=False)]
print("Number of articles after opinion piece filtering:", len(data_filtered))

# Step 4: Remove Irrelevant Content Using Regex
# Define patterns for irrelevant content
irrelevant_patterns = [
    r'All rights reserved', r'Read more', r'For more information', r'Follow us', 
    r'Find us on', r'Contact the author', r'Subscribe for updates', r'\bWATCH\b',
    r'Advertisement', r'^[\W_]+$'  # Removes lines that are mostly symbols or whitespace
]

def clean_irrelevant_content(text):
    for pattern in irrelevant_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return text

data_filtered['content'] = data_filtered['content'].apply(clean_irrelevant_content)
print("Number of articles after irrelevant content cleaning:", len(data_filtered))

# Final DataFrame after filtering
print("Number of articles after all filtering:", len(data_filtered))
data_filtered.head()
data_filtered.info()


Number of articles before filtering: 954
Number of articles after political keyword filtering: 919
Number of articles after opinion piece filtering: 820
Number of articles after irrelevant content cleaning: 820
Number of articles after all filtering: 820
<class 'pandas.core.frame.DataFrame'>
Index: 820 entries, 0 to 952
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   source   820 non-null    object
 1   title    820 non-null    object
 2   content  820 non-null    object
 3   date     820 non-null    object
dtypes: object(4)
memory usage: 32.0+ KB


In [127]:
data_filtered['source'].value_counts()

source
The Daily Caller    94
Breitbart           92
CNN                 85
AP                  85
Zerohedge           83
News Max            83
Guardian            82
Natural News        78
NPR                 77
BBC                 61
Name: count, dtype: int64

# Feature Selection
Here we select the relevant features for fake news classification.
- `title`, `text`
- Create a new DataFrame (`data`) by selecting the specifc columns mentioned above from the original DataFrame `data_raw`.

In [128]:
data = data_filtered[['title', 'content', 'source',]]
print(type(data))
print(data.head())

# Shape before dropping duplicates
print("\nThe old shape is: ", data.shape)

<class 'pandas.core.frame.DataFrame'>
                                               title  \
0  Russia Practices 'Massive Nuclear Strike' in A...   
1  Nolte: Trump and Republican Senate Challenger ...   
2  Sen. Tom Cotton Barnstorms Nation Campaigning ...   
3  U.S. Marines Successfully Test Israel's Iron D...   
4  Seoul, Japan Warn North Korea Preparing Nuclea...   

                                             content     source  
0  “Important to have modern and constantly ready...  Breitbart  
1  The final Insider Advantage poll out of Wiscon...  Breitbart  
2  Sen. Tom Cotton (R-AR) is putting in long hour...  Breitbart  
3  The U.S. Marines have successfully tested a mo...  Breitbart  
4  The South Korean Defense Intelligence Agency (...  Breitbart  

The old shape is:  (820, 3)


# Data Cleaning

## Remove Duplicate Rows
- Drop duplicate rows from the dataframe (`data`).

In [129]:
data_ = data.drop_duplicates()

# Display the new dataframe shape
print("The new shape is: ", data.shape)

The new shape is:  (820, 3)


## Remove Outliers

### `text`

The `text` column of `data`, which is of string type, may contain values with unusually long lengths, indicating the presence of outliers. We will identify the outliers using [Z-score method].

1. Create a new column `text_length` in the DataFrame `data` by calculating the length of each review. (Set the value as 0 if the correponding `text` column has NaN values.)

2. Check the statistics of `text_length` using `describe()` method.

3. Calculate the mean and standard deviation of the `text_length` column.

4. Set the Z-score threshold for identifying outliers to 3.

5. Identify outliers of the `text_length` column and set the corresponding `text` to np.nan.

6. Drop the `text_length` column from the DataFrame.

In [130]:
data['text_length'] = data['content'].apply(lambda x: len(x) if pd.notna(x) else 0)
print(data.head(3))

TL = data["text_length"]
stats_TL = TL.describe()
print(stats_TL)

                                               title  \
0  Russia Practices 'Massive Nuclear Strike' in A...   
1  Nolte: Trump and Republican Senate Challenger ...   
2  Sen. Tom Cotton Barnstorms Nation Campaigning ...   

                                             content     source  text_length  
0  “Important to have modern and constantly ready...  Breitbart         3745  
1  The final Insider Advantage poll out of Wiscon...  Breitbart         2703  
2  Sen. Tom Cotton (R-AR) is putting in long hour...  Breitbart         3989  
count      820.000000
mean      4719.351220
std       3312.200252
min        115.000000
25%       2642.500000
50%       3744.500000
75%       6137.750000
max      24192.000000
Name: text_length, dtype: float64


In [131]:
mean_TL = TL.mean()
# print(mean_TL)

sd_TL = TL.std()
# print(sd_TL)

threshold = 3

z_score = zscore(TL)
# print(z_score)

# Remove 'text' of lengths that are greater than 3 standard deviations above the mean
data.loc[abs(z_score) > threshold, 'content'] = np.nan
# print(data.head(3))

data = data.drop("text_length", axis=1)
data.head()

,title,content,source
0,Russia Practices 'Massive Nuclear Strike' in A...,“Important to have modern and constantly ready...,Breitbart
1,Nolte: Trump and Republican Senate Challenger ...,The final Insider Advantage poll out of Wiscon...,Breitbart
2,Sen. Tom Cotton Barnstorms Nation Campaigning ...,Sen. Tom Cotton (R-AR) is putting in long hour...,Breitbart
3,U.S. Marines Successfully Test Israel's Iron D...,The U.S. Marines have successfully tested a mo...,Breitbart
4,"Seoul, Japan Warn North Korea Preparing Nuclea...",The South Korean Defense Intelligence Agency (...,Breitbart


### `title`

Similarly, the `title` column of `data` (of type `str`) may also contain values with unusually long lengths, indicating the presence of outliers.

1. Create a new column `title_length` in the DataFrame `data` by calculating the length of each price value. (Set the value as 0 if the correponding `title` column has NaN values.)

2. Check the statistics of `title_length` using `describe()` method and display its unique values.

3. Identify the outlier values by inspecting the content in `title` corresponding to the abnormal value in `title_length` and set the corresponding value of `title` to np.nan.

4. Drop the `title_length` column from the DataFrame.

In [132]:
data['title_length'] = data['title'].apply(lambda x: len(x) if pd.notna(x) else 0)
print(data.head(3))

TL = data["title_length"]
stats_TL = TL.describe()
print(stats_TL)

                                               title  \
0  Russia Practices 'Massive Nuclear Strike' in A...   
1  Nolte: Trump and Republican Senate Challenger ...   
2  Sen. Tom Cotton Barnstorms Nation Campaigning ...   

                                             content     source  title_length  
0  “Important to have modern and constantly ready...  Breitbart            72  
1  The final Insider Advantage poll out of Wiscon...  Breitbart            63  
2  Sen. Tom Cotton (R-AR) is putting in long hour...  Breitbart            89  
count    820.000000
mean      79.887805
std       23.841962
min        9.000000
25%       63.000000
50%       79.000000
75%       94.000000
max      186.000000
Name: title_length, dtype: float64


In [133]:
mean_TL = TL.mean()
# print(mean_TL)

sd_TL = TL.std()
# print(sd_TL)

threshold = 3

z_score = zscore(TL)
# print(z_score)

# Remove 'title' of lengths that are greater than 3 standard deviations above the mean
data.loc[abs(z_score) > threshold, 'title'] = np.nan
# print(data.head(3))

data = data.drop("title_length", axis=1)
data.head()

,title,content,source
0,Russia Practices 'Massive Nuclear Strike' in A...,“Important to have modern and constantly ready...,Breitbart
1,Nolte: Trump and Republican Senate Challenger ...,The final Insider Advantage poll out of Wiscon...,Breitbart
2,Sen. Tom Cotton Barnstorms Nation Campaigning ...,Sen. Tom Cotton (R-AR) is putting in long hour...,Breitbart
3,U.S. Marines Successfully Test Israel's Iron D...,The U.S. Marines have successfully tested a mo...,Breitbart
4,"Seoul, Japan Warn North Korea Preparing Nuclea...",The South Korean Defense Intelligence Agency (...,Breitbart


In [134]:
data.isnull().sum()

title       5
content    15
source      0
dtype: int64

# Feature Engineering

### Create new column `full_review`
Since there are some rows with empty `text` and `title`, we will concatenate both columns (`text` and `title`) to form a new column `full_content`.
1. Replace `NaN` values in `text` and `title` with an empty string.

2. Combine `text` and `title` into `full_content`.

3. Strip any leading/trailing whitespaces in `full_content`.

4. Drop `text` and `title` columns.

In [135]:
# 1) Fill NaN values in 'text' and 'title' with an empty string
data['title'] = data['title'].fillna('')
data['content'] = data['content'].fillna('')

# 2) Combine 'text' and 'title' into 'content'
data['full_content'] = data['content'] + " " + data['title']

# 3) Strip any leading/trailing whitespace
data['full_content'] = data['full_content'].str.strip()

# 4) Drop `text` and `title` columns
data = data.drop(columns = ['content', 'title'])

# Check if the 'full_review' column was added and if 'text' and 'title' columns has been dropped
print(data.head())
print("\nThe old shape is:",data.shape)

      source                                       full_content
0  Breitbart  “Important to have modern and constantly ready...
1  Breitbart  The final Insider Advantage poll out of Wiscon...
2  Breitbart  Sen. Tom Cotton (R-AR) is putting in long hour...
3  Breitbart  The U.S. Marines have successfully tested a mo...
4  Breitbart  The South Korean Defense Intelligence Agency (...

The old shape is: (820, 2)


### Handle Missing Values
1. Drop rows where `full_review` are empty strings and reset the index.

2. Check if there are no more null values in `data`.

In [136]:
# 1) Drop rows where `full_review` are empty strings and reset the index
data = data[data['full_content'] != ""].reset_index(drop=True)
print("The new shape is:",data.shape)

# 2) Check if there are no more null values in `data`
data.isnull().sum()

The new shape is: (820, 2)


source          0
full_content    0
dtype: int64

In [137]:
data

,source,full_content
0,Breitbart,“Important to have modern and constantly ready...
1,Breitbart,The final Insider Advantage poll out of Wiscon...
2,Breitbart,Sen. Tom Cotton (R-AR) is putting in long hour...
3,Breitbart,The U.S. Marines have successfully tested a mo...
4,Breitbart,The South Korean Defense Intelligence Agency (...
...,...,...
815,Guardian,A California “home” for rent is shining a ligh...
816,Guardian,Even though the United States touts its status...
817,Guardian,When the Reserve Bank board meet next week to ...
818,Guardian,"If elected, Donald Trump plans to carry out “t..."


# Text Preprocessing for NLP

Here we will define a function `process_full_review` that takes a textual value as input and applies the following processing steps in sequence:

1. Convert the input text to lowercase using the `lower()` function.

2. Tokenize the lowercase text using the `word_tokenize` function from the NLTK library.

3. Create a list (`alphabetic_tokens`) containing only alphanetic tokens using a list comprehension with a regular expression match.

4. Remove stopwords
-   Obtain a set of English stopwords using the `stopwords.words('english')` method.
-   Define a list of `allowed_words` that should not be removed.
-   Remove the stopwords (excluding those that should not be removed).

5. Apply stemming to each token in the list (`lemmatized_words`) using the `lemmatize` method.

6. Join the stemmed tokens into a single processed text using the `join` method and return the processed text.

Create  new columns (`processed_full_review`) in `data` by applying the `process_full_review` function to the `full_review` column.

In [ ]:
# Ensure require NLTK data is downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('all')

In [139]:
# Define function to process text
import string
from nltk.stem import *
from nltk.stem.porter import *

def process_full_review(text):
    # Convert to lowercase and tokenize
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in string.punctuation]
    stemmer = PorterStemmer()
    # List of stopwords
    stop_words = stopwords.words('english')
    allowed_words = ["no", "not", "don't", "dont", "don", "but", 
                     "however", "never", "wasn't", "wasnt", "shouldn't",
                     "shouldnt", "mustn't", "musnt"]

    stemmed = [stemmer.stem(word) for word in tokens if word not in stop_words or word in allowed_words]
    return ' '.join(stemmed)

### <span style="color:red">The code below will take approximately 16 minutes to run!</span>

In [140]:
# Enable tqdm for pandas (progress bar)
tqdm.pandas(desc="Processing News Articles")

# Apply process_full_review function with tqdm progress bar and expand the results into separate columns.
processed_columns = ['processed_full_content']
data[processed_columns] = data['full_content'].progress_apply(lambda x: pd.Series(process_full_review(x)))

data

Processing News Articles: 100%|██████████| 820/820 [00:05<00:00, 160.73it/s]


,source,full_content,processed_full_content
0,Breitbart,“Important to have modern and constantly ready...,“ import modern constantli ready-to-us strateg...
1,Breitbart,The final Insider Advantage poll out of Wiscon...,final insid advantag poll wisconsin show donal...
2,Breitbart,Sen. Tom Cotton (R-AR) is putting in long hour...,sen. tom cotton r-ar put long hour elbow greas...
3,Breitbart,The U.S. Marines have successfully tested a mo...,u.s. marin success test modifi version israel ...
4,Breitbart,The South Korean Defense Intelligence Agency (...,south korean defens intellig agenc dia reporte...
...,...,...,...
815,Guardian,A California “home” for rent is shining a ligh...,california “ home ” rent shine light us hous c...
816,Guardian,Even though the United States touts its status...,even though unit state tout statu one world ’ ...
817,Guardian,When the Reserve Bank board meet next week to ...,reserv bank board meet next week discuss inter...
818,Guardian,"If elected, Donald Trump plans to carry out “t...",elect donald trump plan carri “ largest deport...


In [ ]:
data['source'].value_counts() 

source
The Daily Caller    94
Breitbart           92
CNN                 85
AP                  85
Zerohedge           83
News Max            83
Guardian            82
Natural News        78
NPR                 77
BBC                 61
Name: count, dtype: int64

In [145]:
# Sample 61 articles from each source without including the grouping column in the operation
sample_size = 61

data_sampled = (
    data_filtered.groupby('source', group_keys=False)
    .apply(lambda x: x.sample(n=sample_size, random_state=42))
    .reset_index(drop=True)
)

# Display the result
print("Number of articles after sampling:", len(data_sampled))
print(data_sampled['source'].value_counts())  # Check the counts per source
data_sampled.head()


Number of articles after sampling: 610
source
AP                  61
BBC                 61
Breitbart           61
CNN                 61
Guardian            61
NPR                 61
Natural News        61
News Max            61
The Daily Caller    61
Zerohedge           61
Name: count, dtype: int64


C:\Users\Admin\AppData\Local\Temp\ipykernel_13068\4190293810.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=sample_size, random_state=42))


,source,title,content,date
0,AP,How will abortion laws motivate women in North...,How will abortion laws motivate women in North...,24/10/2024
1,AP,"Who can vote in US elections, and what steps m...","Who can vote in US elections, and what steps m...",8/10/2024
2,AP,Voters drowning in ads from 'obscene' amounts ...,Voters drowning in ads from ‘obscene’ amounts ...,29/10/2024
3,AP,Don't count on a recount to change the winner ...,Don’t count on a recount to change the winner ...,8/10/2024
4,AP,"In battleground Georgia, some poor people see ...","In battleground Georgia, some poor people see ...",22/10/2024
